## Step 1: Importing Packages from Global Python Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

## Step 2: Reading the Data Informed in .csv Format and Turning Into a Dataframe.

In [4]:
raw_data = pd.read_csv('data.csv')
df = pd.DataFrame(raw_data)

## Step 3: Preparing our Data For Classification
#### Before that We Have to Select Targets & Inputs.
#### For that Reason We Have to Create Numeric Type of Migraines for DeepLearning Process

In [5]:
x = df[['Age','Duration','Frequency','Location','Character','Intensity','Nausea','Vomit','Phonophobia','Photophobia','Visual','Sensory','Dysphasia','Dysarthria','Vertigo','Tinnitus','Hypoacusis','Diplopia','Defect','Ataxia','Conscience','Paresthesia','DPF']].values
y = df[['Type']].values

basew, trainglew, altered = [], {}, []

for i in list(df['Type']):
    basew.append(i)

basew = list(set(basew))

for i in basew:
    trainglew[i] = basew.index(i)

for i in list(df['Type']):
    altered.append(trainglew[i])

df['Numeric Type'] = altered
alter_y = df[['Numeric Type']].values

## Step 4: Splitting Our Model Into Train & Test so We Can Conclude that Learning Process will Start from Itself Loop.

In [7]:
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.5)

# Step 4.1: For alter_y, we have to open up another splitting branch.
alter_x_tr, alter_x_ts, alter_y_tr, alter_y_ts = train_test_split(x, alter_y, test_size=0.5)

# Step 5: Using Scaling Algorithms Into It.

In [8]:
sc = StandardScaler()

x_train = sc.fit_transform(x_tr)
x_test = sc.transform(x_ts)

## Step 6: Before We Go Ahead, our Data Structure has Finished that We All Going to Use Same Train and Test Datas for Many More Different ML Models.

#### Technique 1: LOGISTIC REGRESSION with 10-fold cross-validation.

In [9]:
LR = LogisticRegressionCV(cv=10, random_state=17, max_iter=5000)

LR.fit(x_train,y_tr)
lr_pr = LR.predict(x_test)

#### Technique 2: ARTIFICIAL NEURAL NETWORKS (ANN)

In [10]:
print('\nARTIFICIAL NEURAL NETWORKS (ANN)\n--------------------------------')

var_epoch = 10
tf.random.set_seed(17)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='softmax')
])

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(x_train, alter_y_tr, epochs=var_epoch)
print('\nARTIFICIAL NEURAL NETWORKS (ANN) MODEL SUMMARY')
model.summary()

## Let's Visualize the How did ANN Worked out the Classification Thing?

In [11]:
rcParams['figure.figsize'] = (10, 6)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

plt.plot(np.arange(1, var_epoch+1), history.history['loss'], label='Loss')
plt.plot(np.arange(1, var_epoch+1), history.history['categorical_accuracy'], label='Accuracy')
plt.plot(np.arange(1, var_epoch+1), history.history['precision'], label='Precision')
plt.title('Evaluation Metrics', size=20)
plt.xlabel('Epoch', size=15)
plt.legend()
plt.show()

## Step 6: Measuring & Outputting the Metrics.
#### Outputting the Results in Dataframes >>>
#### We are Going to Compare between Predicted Results and Tested Results.

In [12]:
print('\nLOGISTIC REGRESSION >>>\n--------------------------------')

lr_tested = []
for i in list(y_ts):
    lr_tested.append(i[0])

LR_COMPARE = pd.DataFrame()
LR_COMPARE['PREDICTED'] = list(lr_pr)
LR_COMPARE['TESTED'] = lr_tested

print('\n',LR_COMPARE)

print(f'Logistic Regression Accuracy Score: {round(accuracy_score(y_ts,lr_pr),5)}')
print(f'Logistic Regression Precision Score: {round(precision_score(y_ts,lr_pr,average="macro",zero_division=0),5)}')
print(f'Logistic Regression Recall Score: {round(recall_score(y_ts,lr_pr,average="macro"),5)}')